<a href="https://colab.research.google.com/github/aravind-sundaresan/yoda-english-nmt/blob/master/english_to_yoda_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive




**Data Preparation**

In [3]:
# Creating 2 dictionaries each (word-index and index-word) for the source and target languages

with open('/drive/My Drive/data/english_vocabulary.txt', 'r') as f:
  words = f.readlines()

english_index_to_word_dict = dict([(index, word.rstrip('\n')) for index, word in enumerate(words)])
english_word_to_index_dict = dict([(word.rstrip('\n'), index) for index, word in enumerate(words)])

with open('/drive/My Drive/data/yoda_vocabulary.txt', 'r') as f:
  words = f.readlines()

yoda_index_to_word_dict = dict([(index, word.rstrip('\n')) for index, word in enumerate(words)])
yoda_word_to_index_dict = dict([(word.rstrip('\n'), index) for index, word in enumerate(words)])

# Obtaining the number of unique tokens in each vocabulary
english_vocab_length = len(english_word_to_index_dict)
yoda_vocab_length = len(yoda_word_to_index_dict)

'''
Limiting the lengths of the sequences (in terms of number of words) in both the source and target languages 
For source language, max. length = 15 (97% of the sentences have length <= 15)
For target language, max. length = 20 (97% of the sentence have length <= 20)
'''
max_length_source, max_length_target = 15, 20 

english_sentences, yoda_english_sentences = [], []
with open('/drive/My Drive/data/english_sentences.txt', 'r') as fp: 
  line = fp.readline()
  while line:
    line = line.rstrip("\n")
    english_sentences.append([int(token) for token in line.split(" ")])
    line = fp.readline()

with open('/drive/My Drive/data/yoda_english_sentences.txt', 'r') as fp:
  line = fp.readline()
  while line:
    # Adding the start and end tokens to the target sentences
    line = "1 " + line.rstrip("\n") + " 2"
    yoda_english_sentences.append([int(token) for token in line.split(" ")])
    line = fp.readline() 


# Padding the source and target sentences to ensure that all of them have the same length
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(english_sentences, maxlen=max_length_source, padding='post')
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(yoda_english_sentences, maxlen=max_length_target, padding='post')

# Splitting the data into training, validation/dev and test sets
encoder_input_train, encoder_input_test, decoder_input_train, decoder_input_test = train_test_split(encoder_input_data[:115200], decoder_input_data[:115200], test_size=0.1)
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val = train_test_split(encoder_input_train, decoder_input_train, test_size=0.1)


In [4]:
def generate_batch(X, y, batch_size=256):
    # Function to generate a batch of training examples using a generator object because the dataset is too large to fit into memory
    while True:
        for i in range(0, len(X), batch_size):            
            encoder_input_data = X[i:i+batch_size]
            decoder_input_data = y[i:i+batch_size]
            
            '''
             The output of the decoder uses the one-hot representation of each word in a sentence because the output of the seq2seq model
             is obtained from a softmax unit. 
             The size of the decoder output sentence would be (max_length_target, yoda_vocab_length).
             The decoder output does not start with the "_GO" token. The rest of the content is the same as that of decoder input. 
             So the decoder output can be defined as the decoder input shifted or offset by one timestep.
            '''
            decoder_output_data = np.zeros((batch_size, max_length_target, yoda_vocab_length), dtype='float32')

            for j in range(len(decoder_input_data)):
                for k in range(1, max_length_target):
                    decoder_output_data[j, k-1, decoder_input_data[j, k]] = 1
            
            yield([encoder_input_data, decoder_input_data], decoder_output_data)

**Building the Model**

In [5]:
# Size of word embeddings
embedding_dimensions = 50

encoder_input = Input(shape=(None,))

# Embeddings of the English words fed as input to the encoder network
encoder_embeddings = Embedding(english_vocab_length, embedding_dimensions)(encoder_input)

# Encoder LSTM layer
encoder_lstm = LSTM(embedding_dimensions, return_state=True)
encoder_output, encoder_hidden_state, encoder_cell_state = encoder_lstm(encoder_embeddings)

# The encoder outputs are discarded and only the hidden and cell states of the encoder are retained
encoder_states = [encoder_hidden_state, encoder_cell_state]

# Setting up the decoder. The initial state of the decoder is obtained from the encoder_states.

decoder_input = Input(shape=(None,))

# Embeddings of the Yoda English words fed as input to the decoder network
embedding_layer = Embedding(yoda_vocab_length, embedding_dimensions)
decoder_embeddings = embedding_layer(decoder_input)

# Decoder LSTM layer
decoder_lstm = LSTM(embedding_dimensions, return_sequences=True, return_state=True)

decoder_output, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)

# Using a softmax unit to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(yoda_vocab_length, activation='softmax')
decoder_output = decoder_dense(decoder_output)

# Defining the model
model = tf.keras.Model([encoder_input, decoder_input], decoder_output)
# Compiling the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [6]:
# Training the model
training_set_size = len(encoder_input_train)
validation_set_size = len(encoder_input_val)
batch_size = 128
epochs = 32

model.fit_generator(generator=generate_batch(encoder_input_train, decoder_input_train, batch_size=batch_size),
                    steps_per_epoch=training_set_size//batch_size,
                    epochs=epochs,
                    validation_data=generate_batch(encoder_input_val, decoder_input_val, batch_size=batch_size),
                    validation_steps=validation_set_size//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/32
729/729 [==============================] - 924s 1s/step - loss: 3.8238 - acc: 0.4834 - val_loss: 2.9535 - val_acc: 0.6036
Epoch 2/32
729/729 [==============================] - 923s 1s/step - loss: 2.7798 - acc: 0.6259 - val_loss: 2.6800 - val_acc: 0.6434
Epoch 3/32
729/729 [==============================] - 931s 1s/step - loss: 2.5770 - acc: 0.6529 - val_loss: 2.5317 - val_acc: 0.6617
Epoch 4/32
729/729 [==============================] - 917s 1s/step - loss: 2.4526 - acc: 0.6674 - val_loss: 2.4490 - val_acc: 0.6718
Epoch 5/32
729/729 [==============================] - 938s 1s/step - loss: 2.3631 - acc: 0.6779 - val_loss: 2.3813 - val_acc: 0.6780
Epoch 6/32
729/729 [==============================] - 942s 1s/step - loss: 2.2879 - acc: 0.6873 - val_loss: 2.3111 - val_acc: 0.6893
Epoch 7/32
729/729 [==============================] - 940s 1s/step - loss: 2.2195 - acc: 0.6961 - val_loss: 2.2611 - val_acc: 

In [7]:
# Saving the weights of the trained model
model.save_weights('/drive/My Drive/data/yoda_nmt_weights.h5')

In [45]:
model.load_weights('/drive/My Drive/data/yoda_nmt_weights.h5')

In [8]:
# Model setup for Inference

# Defining the encoder model to obtain the (hidden and cell) states representing the context vector
encoder = Model(encoder_input, encoder_states)

# Defining the decoder model for inference

# Hidden and cells states of the decoder from the previous time step
decoder_cell_state_input = Input(shape=(embedding_dimensions,))
decoder_hidden_state_input = Input(shape=(embedding_dimensions,))
decoder_states_input = [decoder_hidden_state_input, decoder_cell_state_input]

decoder_embeddings_inference = embedding_layer(decoder_input)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_inference_output, decoder_hidden_state_output, decoder_cell_state_output = decoder_lstm(decoder_embeddings_inference, initial_state=decoder_states_input)
decoder_states_output = [decoder_hidden_state_output, decoder_cell_state_output]
decoder_inference_output = decoder_dense(decoder_inference_output)

decoder = Model([decoder_input] + decoder_states_input, [decoder_inference_output] + decoder_states_output)



In [9]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     4907950     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     4547050     input_2[0][0]                    
_______________________________________________________________________________________

In [10]:
# Translation Function

def translate(input_sequence):
    
    # Obtain the states of the encoder after processing the input sequence
    model_states = encoder.predict(input_sequence)
    
    # Initializing the target sequence to store the output and setting the first character as the "Start" token
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = yoda_word_to_index_dict['_GO']
    
    stop_condition = False
    output_sentence = ''
    
    while stop_condition is not True:
        output_tokens, hidden_state, cell_state = decoder.predict([target_sequence] + model_states)
    
        # Sample a token/word with the highest probability from the output of the sigmoid unit
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = yoda_index_to_word_dict[sampled_token_index]
        output_sentence += sampled_token + ' '
        
        # Checking if the translation is done
        if sampled_token == '_EOS' or len(output_sentence) > 50:
            stop_condition = True
        
        # Updating the target sequence
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index
        
        # Updating the states
        model_states = [hidden_state, cell_state]
    
    return output_sentence

In [32]:
train_gen = generate_batch(encoder_input_train, decoder_input_train, batch_size=1)

In [68]:
(input_seq, actual_output), _ = next(train_gen)

english_sentence = ''
for i in input_seq[0]:
  english_sentence += english_index_to_word_dict[i] + ' '

print(english_sentence)

print("##")
english_sentence = ''
for i in actual_output[0]:
  english_sentence += yoda_index_to_word_dict[i] + ' '
print(english_sentence)

print('##')
translated_sentence = translate(input_seq)
print(translated_sentence)

• by the book ; _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 
##
_GO • by the book ; Yeesssssss . _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 
##
• by the company . Herh herh herh . _EOS 


In [31]:
translate(["There is beauty in ordinary things"])

UnimplementedError: ignored

In [30]:
count = 0

for i in range(10):
  english_sentence = ''
  yoda_sentence = ''
  for j in encoder_input_train[i]:
    english_sentence += english_index_to_word_dict[j] + ' '
  print(english_sentence)
  for j in decoder_input_train[i]:
    yoda_sentence += yoda_index_to_word_dict[j] + ' ' 
  print(yoda_sentence)
  print("\n\n")


% chg United States 0 . 0 Denmark Other TOTAL IMPORTS _PAD _PAD _PAD _PAD 
_GO +chg united states 0 . 0 denmark other total imports . Yeesssssss . _EOS _PAD _PAD _PAD _PAD _PAD 



There are no resistant cultivars . _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 
_GO No resistant cultivars , there are . Yeesssssss . _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 



• maintaining common national qualification standards ; _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 
_GO • maintaining common national qualification standards ; Hmmmmmm . _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD 



Number of earlier filing ( if available ) : _PAD _PAD _PAD _PAD _PAD _PAD 
_GO Of earlier filing ( if available ) number : Yeesssssss . _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD 



Four action items fall under " advancing local and global sustainability " . _PAD _PAD 
_GO Four action items fall under " Advancing local and global sustainability " . _EOS _PAD _PAD _PAD _PAD _PAD 



Who Benefits & Who Pays ? 

In [71]:
count = 0
with open('/drive/My Drive/data/english_sentences.txt', 'r') as fp: 
  line = fp.readline()
  while line and count < 15:
    line = line.rstrip("\n")
    tokens = [int(token) for token in line.split(" ")]
    english_sentence = ""
    for token in tokens:
      english_sentence += english_index_to_word_dict[token] + ' '
    
    print(english_sentence)


    # english_sentences.append()
    line = fp.readline()
    count += 1

print("##")

count = 0
with open('/drive/My Drive/data/yoda_english_sentences.txt', 'r') as fp: 
  line = fp.readline()
  while line and count < 15:
    line = line.rstrip("\n")
    tokens = [int(token) for token in line.split(" ")]
    english_sentence = ""
    for token in tokens:
      english_sentence += yoda_index_to_word_dict[token] + ' '
    
    print(english_sentence)


    # english_sentences.append()
    line = fp.readline()
    count += 1
# with open('/drive/My Drive/data/yoda_english_sentences.txt', 'r') as fp:
#   line = fp.readline()
#   while line:
#     # Adding the start and end tokens to the target sentences
#     line = "1 " + line.rstrip("\n") + " 2"
#     yoda_english_sentences.append([int(token) for token in line.split(" ")])
#     line = fp.readline() 

What is light ? 
Astronomers Introduction Introduction video What is Astronomy ? 
It prompts us to ask the deepest existential questions : 
Who are we ? 
Where did we come from ? 
Are we alone ? 
Today , astronomy covers several disciplines : 
Verbatim A journey into Canadian astronomy 
Observatories Jacques Cartier explores the continental interior . 
Instruments Leonard Digges constructs the first telescope using lenses . 
Observatories Jesuits record the first astronomical observations made in Canada . 
Instruments Isaac Newton builds his telescope . 
Instruments French astronomer Dominique François Jean Arago invents the polariscope . 
Instruments British astronomer John Frederick William Herschel invents the photometer . 
Astronomes William Frederick King is born . 
##
Light , what is , hmm ? 
Astronomy , astronomers introduction introduction video what is , hmm ? Yes , hmmm . 
To ask the deepest existential questions it prompts us : 
We , who are , hmm ? Yeesssssss . 
From , wher